In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [2]:
from mlxtend.preprocessing import minmax_scaling 

In [3]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.11.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,102.5,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,32.0,169.5,30.1,0.349,47,1


In [5]:
scaled_data = minmax_scaling(df,columns=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'])

In [6]:
def build_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=[len(scaled_data.keys())]),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.01)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = build_model()

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [8]:
EPOCHS = 1000

history = model.fit(scaled_data, df['Outcome'],epochs=EPOCHS, validation_split=0.2, verbose=2)

Epoch 1/1000
20/20 - 8s - loss: 0.6599 - accuracy: 0.6531 - val_loss: 0.6443 - val_accuracy: 0.6429 - 8s/epoch - 414ms/step
Epoch 2/1000
20/20 - 0s - loss: 0.6301 - accuracy: 0.6531 - val_loss: 0.6313 - val_accuracy: 0.6429 - 191ms/epoch - 10ms/step
Epoch 3/1000
20/20 - 0s - loss: 0.5977 - accuracy: 0.6531 - val_loss: 0.6165 - val_accuracy: 0.6429 - 278ms/epoch - 14ms/step
Epoch 4/1000
20/20 - 0s - loss: 0.5634 - accuracy: 0.6694 - val_loss: 0.5369 - val_accuracy: 0.7662 - 288ms/epoch - 14ms/step
Epoch 5/1000
20/20 - 0s - loss: 0.5259 - accuracy: 0.7394 - val_loss: 0.5188 - val_accuracy: 0.7727 - 311ms/epoch - 16ms/step
Epoch 6/1000
20/20 - 0s - loss: 0.4870 - accuracy: 0.7704 - val_loss: 0.5737 - val_accuracy: 0.6818 - 212ms/epoch - 11ms/step
Epoch 7/1000
20/20 - 0s - loss: 0.4664 - accuracy: 0.7785 - val_loss: 0.5553 - val_accuracy: 0.6818 - 323ms/epoch - 16ms/step
Epoch 8/1000
20/20 - 0s - loss: 0.4640 - accuracy: 0.7655 - val_loss: 0.4734 - val_accuracy: 0.7662 - 214ms/epoch - 11ms

In [9]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
995,0.193466,0.936482,0.543878,0.831169,995
996,0.206867,0.928339,0.524164,0.837662,996
997,0.201144,0.923453,0.504662,0.857143,997
998,0.199624,0.933225,0.523637,0.837662,998
999,0.195551,0.926710,0.583159,0.831169,999


In [10]:
acc = (hist['accuracy'].tail().sum())*100/5 
val_acc = (hist['val_accuracy'].tail().sum())*100/5 

print("Training Accuracy = {}% and Validation Accuracy = {}%".format(round(acc, 2), round(val_acc, 2)))

Training Accuracy = 92.96% and Validation Accuracy = 83.9%
